In [1]:
pip install torchmetrics 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.

In [2]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
import time
import copy
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import torchmetrics

# Install torchinfo if not present (uncomment in a notebook)
# !pip install torchinfo
from torchinfo import summary

# Classification Task

### Configuration

In [4]:
# --- 1. Configuration Constants ---
IMG_SIZE = 224   # Input size required by DenseNet121
BATCH_SIZE = 32
NUM_CLASSES = 4  # Target classes: glioma, meningioma, no_tumor, pituitary
SEED = 42

In [5]:
# Hyperparameters
NUM_WORKERS = 4
NUM_EPOCHS_HEAD = 5
NUM_EPOCHS_FINE_TUNE = 15
LEARNING_RATE_HEAD = 0.001
LEARNING_RATE_FINE_TUNE = 1e-5   

In [6]:
# Paths
BASE_PATH = '/kaggle/input/brisc2025/brisc2025/classification_task'
TRAIN_PATH = f'{BASE_PATH}/train'
TEST_PATH = f'{BASE_PATH}/test'

In [7]:
# Device Setup
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

Using device: cuda


In [8]:
# Set seed for reproducibility
torch.manual_seed(SEED)

### Data Preparation

In [9]:
# Standard normalization required for ImageNet-pretrained models
NORM_MEAN = [0.485, 0.456, 0.406]
NORM_STD = [0.229, 0.224, 0.225]

In [10]:
# Transformations for training (with augmentation) and validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(IMG_SIZE), 
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),                  
        transforms.Normalize(NORM_MEAN, NORM_STD) 
    ]),
    'val_test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(NORM_MEAN, NORM_STD)
    ]),
}

In [11]:
# Load the full dataset and split it
full_train_dataset = datasets.ImageFolder(TRAIN_PATH)
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_data, val_data = random_split(full_train_dataset, [train_size, val_size])

In [12]:
# Apply the appropriate transforms
train_data.dataset.transform = data_transforms['train']
val_data.dataset.transform = data_transforms['val_test']

In [13]:
test_data = datasets.ImageFolder(TEST_PATH, data_transforms['val_test'])

In [14]:
# Create DataLoaders for batching and shuffling
dataloaders = {
    'train': DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS),
    'val': DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS),
    'test': DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS), 
}

In [15]:
dataset_sizes = {
    'train': len(train_data), 
    'val': len(val_data),
    'test': len(test_data)
}

In [16]:
class_names = full_train_dataset.classes
print(f"Classes found: {class_names}")

Classes found: ['glioma', 'meningioma', 'no_tumor', 'pituitary']


### Model Options

#### Vision Transformer

In [17]:

# model = models.vit_b_16(weights=None)
# ViT-B/16 (PRE-TRAINED)

model = models.vit_b_16(weights=models.ViT_B_16_Weights.DEFAULT)
model.heads.head = nn.Linear(model.heads.head.in_features, 4)

Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:01<00:00, 236MB/s] 


#### Swin Transformer

In [ ]:
# model = models.swin_t(weights=None)
# Swin Transformer Tiny (PRE-TRAINED) 

model = models.swin_t(weights=models.Swin_T_Weights.DEFAULT)
model.head = nn.Linear(model.head.in_features, 4)

#### DenseNet121

In [ ]:
# DenseNet121 (NO PRE-TRAINING) 
model = models.densenet121(weights=None) # <-

In [ ]:
# DenseNet121 (PRE-TRAINED) 
model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)

In [ ]:
# Replace the final classification layer (model.classifier in DenseNet)
num_ftrs = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_ftrs, NUM_CLASSES),
    nn.Softmax(dim=1) 
)

##### EfficientNetV2

In [ ]:
# EfficientNetV2 (NO PRE-TRAINING) 
model = models.efficientnet_v2_s(weights=None)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 4)

In [ ]:
# EfficientNetV2 (PRE-TRAINED) 
model = models.efficientnet_v2_s(weights=models.EfficientNet_V2_S_Weights.DEFAULT)
model.classifier[1] = nn.Linear(model_eff.classifier[1].in_features, 4)

##### ConvNeXt

In [24]:
# ConvNeXt (NO PRE-TRAINING) 
model = models.convnext_tiny(weights=None)
model.classifier[2] = nn.Linear(model.classifier[2].in_features, 4)

In [ ]:
# ConvNeXt (PRE-TRAINED) 
model = models.convnext_tiny(weights=models.ConvNeXt_Tiny_Weights.DEFAULT)
model.classifier[2] = nn.Linear(model_conv.classifier[2].in_features, 4)

##### ResNet50

In [ ]:
# ResNet50 (NO PRE-TRAINING) 
model_res = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

In [ ]:
# ResNet50 (PRE-TRAINED) 
model_res = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model_res.fc = nn.Linear(model_res.fc.in_features, 4)

##### Freezing extraction layers

In [19]:
# ONLY RUN WHEN THERE ARE WEIGHTS!!

# Stage 1: Freeze the feature extraction layers
#for param in model.parameters():
#    param.requires_grad = False

### Model Configuration

In [18]:
model = model.to(DEVICE)

In [19]:
summary(
    model, 
    # Use '1' for the batch size instead of '-1'
    input_size=(1, 3, IMG_SIZE, IMG_SIZE), 
    col_names=["input_size", "output_size", "num_params", "trainable"],
    verbose=1
)

Layer (type:depth-idx)                        Input Shape               Output Shape              Param #                   Trainable
VisionTransformer                             [1, 3, 224, 224]          [1, 4]                    768                       True
├─Conv2d: 1-1                                 [1, 3, 224, 224]          [1, 768, 14, 14]          590,592                   True
├─Encoder: 1-2                                [1, 197, 768]             [1, 197, 768]             151,296                   True
│    └─Dropout: 2-1                           [1, 197, 768]             [1, 197, 768]             --                        --
│    └─Sequential: 2-2                        [1, 197, 768]             [1, 197, 768]             --                        True
│    │    └─EncoderBlock: 3-1                 [1, 197, 768]             [1, 197, 768]             7,087,872                 True
│    │    └─EncoderBlock: 3-2                 [1, 197, 768]             [1, 197, 768]         

Layer (type:depth-idx)                        Input Shape               Output Shape              Param #                   Trainable
VisionTransformer                             [1, 3, 224, 224]          [1, 4]                    768                       True
├─Conv2d: 1-1                                 [1, 3, 224, 224]          [1, 768, 14, 14]          590,592                   True
├─Encoder: 1-2                                [1, 197, 768]             [1, 197, 768]             151,296                   True
│    └─Dropout: 2-1                           [1, 197, 768]             [1, 197, 768]             --                        --
│    └─Sequential: 2-2                        [1, 197, 768]             [1, 197, 768]             --                        True
│    │    └─EncoderBlock: 3-1                 [1, 197, 768]             [1, 197, 768]             7,087,872                 True
│    │    └─EncoderBlock: 3-2                 [1, 197, 768]             [1, 197, 768]         

### Training

#### Train Classification Head 

In [22]:
# ADD THIS NEW FUNCTION IN ITS PLACE
def train_model(model, dataloaders, criterion, optimizer, num_epochs):
    """
    Generic PyTorch training loop now with Precision, Recall, and F1.
    """
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # Initialize metric objects for multi-class classification
    metrics = {
        'acc': torchmetrics.Accuracy(task="multiclass", num_classes=NUM_CLASSES).to(DEVICE),
        'precision': torchmetrics.Precision(task="multiclass", num_classes=NUM_CLASSES, average='macro').to(DEVICE),
        'recall': torchmetrics.Recall(task="multiclass", num_classes=NUM_CLASSES, average='macro').to(DEVICE),
        'f1': torchmetrics.F1Score(task="multiclass", num_classes=NUM_CLASSES, average='macro').to(DEVICE)
    }

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            # Reset all metrics at the start of each phase
            for m in metrics.values():
                m.reset()

            running_loss = 0.0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)
                optimizer.zero_grad() 

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward() 
                        optimizer.step() 
                
                # Update metrics with the current batch
                for m in metrics.values():
                    m.update(preds, labels)
                
                running_loss += loss.item() * inputs.size(0)

            # Compute metrics over the entire epoch
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = metrics['acc'].compute()
            epoch_precision = metrics['precision'].compute()
            epoch_recall = metrics['recall'].compute()
            epoch_f1 = metrics['f1'].compute()

            # This print statement now shows all metrics
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} Prec: {epoch_precision:.4f} Rec: {epoch_recall:.4f} F1: {epoch_f1:.4f}')
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    model.load_state_dict(best_model_wts)
    return model

In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [23]:
print("\n[STAGE 1] Training classification head (Base Frozen)...")
criterion = nn.CrossEntropyLoss()
#optimizer_head = optim.Adam(model.classifier.parameters(), lr=LEARNING_RATE_HEAD) #CNNs
optimizer_head = optim.Adam(model.heads.head.parameters(), lr=LEARNING_RATE_HEAD) # ViTs
model = train_model(model, dataloaders, criterion, optimizer_head, NUM_EPOCHS_HEAD)


[STAGE 1] Training classification head (Base Frozen)...
Epoch 0/4
----------
train Loss: 0.5288 Acc: 0.8280 Prec: 0.8399 Rec: 0.8280 F1: 0.8310
val Loss: 0.3555 Acc: 0.8910 Prec: 0.9030 Rec: 0.8853 F1: 0.8906
Epoch 1/4
----------
train Loss: 0.2958 Acc: 0.9070 Prec: 0.9123 Rec: 0.9075 F1: 0.9088
val Loss: 0.2840 Acc: 0.8970 Prec: 0.8992 Rec: 0.8947 F1: 0.8953
Epoch 2/4
----------
train Loss: 0.2399 Acc: 0.9235 Prec: 0.9274 Rec: 0.9244 F1: 0.9254
val Loss: 0.2407 Acc: 0.9160 Prec: 0.9134 Rec: 0.9149 F1: 0.9137
Epoch 3/4
----------
train Loss: 0.2059 Acc: 0.9350 Prec: 0.9378 Rec: 0.9356 F1: 0.9363
val Loss: 0.2184 Acc: 0.9270 Prec: 0.9270 Rec: 0.9241 F1: 0.9253
Epoch 4/4
----------
train Loss: 0.1846 Acc: 0.9398 Prec: 0.9418 Rec: 0.9406 F1: 0.9410
val Loss: 0.2023 Acc: 0.9370 Prec: 0.9380 Rec: 0.9345 F1: 0.9359
Training complete in 14m 36s
Best val Acc: 0.937000


#### Fine-tuning

In [24]:
print("\n[STAGE 2] Fine-tuning the entire model (Unfrozen)...")
for param in model.parameters():
    param.requires_grad = True
    
optimizer_fine = optim.Adam(model.parameters(), lr=LEARNING_RATE_FINE_TUNE)
model = train_model(model, dataloaders, criterion, optimizer_fine, NUM_EPOCHS_FINE_TUNE)


[STAGE 2] Fine-tuning the entire model (Unfrozen)...
Epoch 0/14
----------
train Loss: 0.1276 Acc: 0.9563 Prec: 0.9578 Rec: 0.9574 F1: 0.9575
val Loss: 0.0735 Acc: 0.9720 Prec: 0.9730 Rec: 0.9720 F1: 0.9723
Epoch 1/14
----------
train Loss: 0.0224 Acc: 0.9952 Prec: 0.9954 Rec: 0.9953 F1: 0.9954
val Loss: 0.0375 Acc: 0.9860 Prec: 0.9858 Rec: 0.9859 F1: 0.9859
Epoch 2/14
----------
train Loss: 0.0047 Acc: 0.9998 Prec: 0.9998 Rec: 0.9998 F1: 0.9998
val Loss: 0.0315 Acc: 0.9890 Prec: 0.9882 Rec: 0.9897 F1: 0.9889
Epoch 3/14
----------
train Loss: 0.0042 Acc: 0.9995 Prec: 0.9996 Rec: 0.9996 F1: 0.9996
val Loss: 0.0349 Acc: 0.9840 Prec: 0.9839 Rec: 0.9838 F1: 0.9838
Epoch 4/14
----------
train Loss: 0.0013 Acc: 1.0000 Prec: 1.0000 Rec: 1.0000 F1: 1.0000
val Loss: 0.0287 Acc: 0.9860 Prec: 0.9861 Rec: 0.9858 F1: 0.9859
Epoch 5/14
----------
train Loss: 0.0007 Acc: 1.0000 Prec: 1.0000 Rec: 1.0000 F1: 1.0000
val Loss: 0.0252 Acc: 0.9920 Prec: 0.9917 Rec: 0.9926 F1: 0.9922
Epoch 6/14
----------


In [1]:
print("\nPyTorch training complete.")


PyTorch training complete.


### Evaluation

In [30]:
# ADD THIS FINAL VERSION
import torchmetrics
from sklearn.metrics import classification_report

def evaluate_model(model, test_loader, class_names): 
    model.eval() 
    running_loss = 0.0
    
    # --- Initialize all metric objects ---
    acc_metric = torchmetrics.Accuracy(task="multiclass", num_classes=NUM_CLASSES).to(DEVICE)
    prec_metric = torchmetrics.Precision(task="multiclass", num_classes=NUM_CLASSES, average='macro').to(DEVICE)
    rec_metric = torchmetrics.Recall(task="multiclass", num_classes=NUM_CLASSES, average='macro').to(DEVICE)
    spec_metric = torchmetrics.Specificity(task="multiclass", num_classes=NUM_CLASSES, average='macro').to(DEVICE)
    auroc_metric = torchmetrics.AUROC(task="multiclass", num_classes=NUM_CLASSES).to(DEVICE)
    
    all_preds = []
    all_labels = []
    
    # Make sure criterion is defined
    criterion = nn.CrossEntropyLoss() 
    
    with torch.no_grad(): 
        for inputs, labels in test_loader:
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            
            # Update metrics
            acc_metric.update(preds, labels)
            prec_metric.update(preds, labels)
            rec_metric.update(preds, labels)
            spec_metric.update(preds, labels)
            auroc_metric.update(outputs, labels) 
            
            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())
            
    # --- Compute all final metrics ---
    test_loss = running_loss / dataset_sizes['test']
    test_acc = acc_metric.compute().item()
    test_prec = prec_metric.compute().item()
    test_rec = rec_metric.compute().item()
    test_auroc = auroc_metric.compute().item()
    test_spec = spec_metric.compute().item()
    
    # --- Get per-class F1 scores ---
    all_preds_tensor = torch.cat(all_preds)
    all_labels_tensor = torch.cat(all_labels)
    
    report_dict = classification_report(
        all_labels_tensor.numpy(), 
        all_labels_tensor.numpy(), 
        target_names=class_names,
        output_dict=True
    )
    
    f1_per_class = {}
    for class_name in class_names:
        f1_per_class[class_name] = report_dict[class_name]['f1-score']

    # --- Create the 'scores' list in the Keras order ---
    scores = []
    scores.append(test_loss)      # scores[0]: Loss
    scores.append(test_acc)       # scores[1]: Accuracy
    scores.append(test_prec)      # scores[2]: Precision (Macro)
    scores.append(test_rec)       # scores[3]: Recall (Macro)
    scores.append(test_auroc)     # scores[4]: AUC (AUROC Macro)
    scores.append(test_spec)      # scores[5]: Specificity (Macro)
    scores.append(f1_per_class)   # scores[6]: F1 Scores per class
    
    # --- THIS IS THE LINE THAT FIXES THE ERROR ---
    return scores

### Network Scores

##### DenseNet121 Results

In [ ]:
scores = evaluate_model(model, dataloaders['test'], class_names)

print("--- DenseNet121 Final Results --- with Pre-trained")
print(f"Test Loss: {scores[0]} \nAccuracy: {scores[1]} \nPrecision: {scores[2]}\nRecall: {scores[3]}\nAUC: {scores[4]}\nSpecificity: {scores[5]}")
print(f"\nF1 SCORES")
for class_name in class_names:
    print(f"{class_name}: {scores[6][class_name]}")

In [ ]:
scores = evaluate_model(model, dataloaders['test'], class_names)

print("--- DenseNet121 Final Results --- No Pre-training")
print(f"Test Loss: {scores[0]} \nAccuracy: {scores[1]} \nPrecision: {scores[2]}\nRecall: {scores[3]}\nAUC: {scores[4]}\nSpecificity: {scores[5]}")
print(f"\nF1 SCORES")
for class_name in class_names:
    print(f"{class_name}: {scores[6][class_name]}")

##### ConvNexT

In [31]:
scores = evaluate_model(model, dataloaders['test'], class_names)

print("--- ConvNext Final Results --- No Pre-training")
print(f"Test Loss: {scores[0]} \nAccuracy: {scores[1]} \nPrecision: {scores[2]}\nRecall: {scores[3]}\nAUC: {scores[4]}\nSpecificity: {scores[5]}")
print(f"\nF1 SCORES")
for class_name in class_names:
    print(f"{class_name}: {scores[6][class_name]}")

--- ConvNext Final Results --- No Pre-training
Test Loss: 0.779368567943573 
Accuracy: 0.6790000200271606 
Precision: 0.6617140769958496
Recall: 0.7012651562690735
AUC: 0.8893592357635498
Specificity: 0.8931803703308105

F1 SCORES
glioma: 1.0
meningioma: 1.0
no_tumor: 1.0
pituitary: 1.0


# Segmentation Task